In [1]:
# default_exp utils

In [2]:
#hide
from nbdev.showdoc import *

In [15]:
# export
from loguru import logger
from functools import wraps

ICACHE = dict()


def imemoize(func):
    """
        Memoize a function into memory, the function recaculate only change when its belonging arguments change
    """
    @wraps(func)
    def _f(*args, **kwargs):
        import mmcv
        timer = mmcv.Timer()
        import inspect
        from avcv.utils import identify
        ident_name = identify((inspect.getsource(func), args, kwargs))
        if not ident_name in ICACHE:
            # logger.warning('{} not in CACHE: "{}"'.format(ident_name, ICACHE))
            result = func(*args, **kwargs)
            logger.info('Imemoize {}, init:  runtime: {:0.2f} s'.format(
                func.__name__, timer.since_last_check()))
            ICACHE[ident_name] = result
        else:
            result = ICACHE[ident_name]
            logger.info('Imemoize {} reuse, runtime: {:0.2f} s'.format(
                func.__name__, timer.since_last_check()))

        return result
    return _f


In [16]:

import time
@imemoize
def example_function(x):
    x = x+1
    time.sleep(5)
    return x

x = example_function(0)
x = example_function(0)

2021-12-25 11:55:07.389 | INFO     | __main__:_f:21 - Imemoize example_function, init:  runtime: 5.01 s
2021-12-25 11:55:07.392 | INFO     | __main__:_f:26 - Imemoize example_function reuse, runtime: 0.00 s


2021-12-25 11:53:07.011 | INFO     | __main__:_f:16 - Imemoize example_function, init:  runtime: 5.01 s
